<a href="https://colab.research.google.com/github/ashiqurrahmankhan21st/BreastCancer/blob/main/Breast_Cancer_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
#import keras_metrics
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import time

import sdv
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer

In [27]:
#df = pd.read_csv("https://raw.githubusercontent.com/ashiqurrahmankhan21st/BreastCancer/main/data.csv")
#del df['id']
#df.head()

In [28]:
def data(x,data_amount):

  data_original = pd.read_csv("https://raw.githubusercontent.com/ashiqurrahmankhan21st/BreastCancer/main/data.csv")
  del data_original['id']
  metadata = SingleTableMetadata()
  metadata.detect_from_dataframe(data=data_original)
  if x == "o":
    df = data_original.copy()

  elif x == 'gc':


    synthesizer = GaussianCopulaSynthesizer(metadata)
    synthesizer.fit(data_original)
    gcs_synthetic_data = synthesizer.sample(num_rows=data_amount)
    gcs_synthetic_data.head()
    df = gcs_synthetic_data.copy()

  elif x== "tvae":
    from sdv.single_table import TVAESynthesizer

    synthesizer = TVAESynthesizer(
      metadata, # required
      enforce_min_max_values=True,
      enforce_rounding=False,
      epochs=500
                )
    synthesizer.fit(data_original)

    tvae_synthetic_data = synthesizer.sample(num_rows=data_amount)
    df = tvae_synthetic_data

  elif x == "F":
    from sdv.lite import SingleTablePreset

    synthesizer = SingleTablePreset(
        metadata, # required
        name='FAST_ML', # required
        locales=['en_US', 'en_CA']
    )
    synthesizer.fit(data_original)

    fastML_synthetic_data = synthesizer.sample(num_rows=data_amount)
    df = fastML_synthetic_data




  encoder = LabelEncoder()
  df['diagnosis'] = encoder.fit_transform(df['diagnosis'])
  X = df.drop(columns=['diagnosis']).copy()
  X = StandardScaler().fit_transform(X).copy()
  y = df.diagnosis.copy()

  return df, X, y

In [29]:
df, X,y = data("tvae",10000)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [30]:
df['diagnosis'].value_counts()

0    6024
1    3976
Name: diagnosis, dtype: int64

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=123)
print("Original data : ",df.shape)
print("tarin         : ",X_train.shape)
print("test          : ",X_test.shape[0])
print("validation    : ",X_val.shape[0])

Original data :  (10000, 31)
tarin         :  (8000, 30)
test          :  1000
validation    :  1000


In [32]:
# SVM
svm = SVC(C=0.1, gamma='auto', kernel = 'rbf')
svm.fit(X_train, y_train)
STr = svm.score(X_train, y_train)
STe = svm.score(X_test, y_test)
y_pred_svm = svm.predict(X_test)

In [33]:
#ANN
tf.random.set_seed(123)
ANNmodel = Sequential()
ANNmodel.add(Dense(30, activation='relu', input_shape=(X_train.shape[1],)))
ANNmodel.add(Dense(10, activation='relu'))
ANNmodel.add(Dense(1, activation='sigmoid'))
ANNmodel.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy']) #tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
ANNmodel.fit(X_train,y_train, batch_size = 128, epochs = 20, validation_data = (X_val,y_val))
ATr = ANNmodel.evaluate(X_train,y_train,verbose=0)[1]
ATe = ANNmodel.evaluate(X_test,y_test,verbose=0)[1]
y_pred_ANN = (ANNmodel.predict(X_test) > 0.5).astype("int32")

Epoch 1/20
63/63 [==============================] - 1s 4ms/step - loss: 0.2774 - accuracy: 0.9065 - val_loss: 0.1676 - val_accuracy: 0.9450
Epoch 2/20
63/63 [==============================] - 0s 2ms/step - loss: 0.1328 - accuracy: 0.9475 - val_loss: 0.1256 - val_accuracy: 0.9520
Epoch 3/20
63/63 [==============================] - 0s 2ms/step - loss: 0.1102 - accuracy: 0.9538 - val_loss: 0.1110 - val_accuracy: 0.9530
Epoch 4/20
63/63 [==============================] - 0s 2ms/step - loss: 0.1015 - accuracy: 0.9575 - val_loss: 0.1050 - val_accuracy: 0.9540
Epoch 5/20
63/63 [==============================] - 0s 2ms/step - loss: 0.0962 - accuracy: 0.9585 - val_loss: 0.1013 - val_accuracy: 0.9570
Epoch 6/20
63/63 [==============================] - 0s 2ms/step - loss: 0.0932 - accuracy: 0.9610 - val_loss: 0.0993 - val_accuracy: 0.9570
Epoch 7/20
63/63 [==============================] - 0s 2ms/step - loss: 0.0903 - accuracy: 0.9617 - val_loss: 0.0972 - val_accuracy: 0.9590
Epoch 8/20
63/63 [==

In [34]:
#XGBoost
params = {
            'objective':'binary:logistic',
            'max_depth': 7,
            'alpha': 10,
            'learning_rate': 1,
            'n_estimators':100
        }
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
XTr = accuracy_score(y_train, xgb.predict(X_train))
XTe = accuracy_score(y_test, xgb.predict(X_test))

In [35]:
XTr,XTe

(0.990875, 0.955)

In [36]:
#KNN
# Define the range of n_neighbors values to test
n_neighbors_values = [1,3, 5, 7, 9, 11]

best_accuracy = 0.0
best_n_neighbors = None

for n_neighbors in n_neighbors_values:
    print("Number of Neighbors:", n_neighbors)

    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)

    y_pred_knn = knn.predict(X_test)

    train_accuracy = accuracy_score(y_train, knn.predict(X_train))
    test_accuracy = accuracy_score(y_test, knn.predict(X_test))

    print('KNN model train accuracy score: {0:0.4f}'.format(train_accuracy))
    print('KNN model test accuracy score: {0:0.4f}'.format(test_accuracy))
    print()

    # Check if the current test accuracy is better than the previous best
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_n_neighbors = n_neighbors
print("best neighbours: ", best_n_neighbors)
knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knn.fit(X_train, y_train)
KTr = accuracy_score(y_train, knn.predict(X_train))
KTe = accuracy_score(y_test, knn.predict(X_test))
y_pred_knn = knn.predict(X_test)


Number of Neighbors: 1
KNN model train accuracy score: 1.0000
KNN model test accuracy score: 0.9410

Number of Neighbors: 3
KNN model train accuracy score: 0.9670
KNN model test accuracy score: 0.9540

Number of Neighbors: 5
KNN model train accuracy score: 0.9615
KNN model test accuracy score: 0.9590

Number of Neighbors: 7
KNN model train accuracy score: 0.9593
KNN model test accuracy score: 0.9600

Number of Neighbors: 9
KNN model train accuracy score: 0.9587
KNN model test accuracy score: 0.9610

Number of Neighbors: 11
KNN model train accuracy score: 0.9574
KNN model test accuracy score: 0.9590

best neighbours:  9


In [37]:
#RF
rf = RandomForestClassifier(n_estimators= 500,max_features = 'sqrt', max_samples = 100, random_state=123)
rf.fit(X_train, y_train)
RTr = accuracy_score(y_train, rf.predict(X_train))
RTe = accuracy_score(y_test, rf.predict(X_test))
y_pred_rf = rf.predict(X_test)

In [38]:
#LR
lr = LogisticRegression(C= 0.1 , penalty='l1', solver='liblinear', max_iter = 1000, random_state=123)
lr.fit(X_train, y_train)
LTr = accuracy_score(y_train, lr.predict(X_train))
LTe = accuracy_score(y_test, lr.predict(X_test))
y_pred_lr = lr.predict(X_test)

In [39]:
acc = pd.DataFrame(
    {
    "SVM":[STr,STe],
    "KNN":[KTr,KTe],
    "RF" :[RTr,RTe],
    "LR" :[LTr,LTe],
    "ANN":[ATr,ATe],
    "XGB":[XTr,XTe] })
acc.index = ["train", "test"]
acc = acc.T
acc

,train,test
SVM,0.958625,0.961
KNN,0.958750,0.961
RF,0.957500,0.962
LR,0.960500,0.960
ANN,0.967500,0.966
XGB,0.990875,0.955


In [40]:
newdata = pd.DataFrame({
    "SVM": y_pred_svm
})
newdata["KNN"] = y_pred_knn
newdata["RF"] = y_pred_rf
newdata['LR'] = y_pred_lr
newdata["ANN"] = y_pred_ANN
newdata["XGB"] = y_pred_xgb
newdata['y_test'] = list(y_test)
newdata

,SVM,KNN,RF,LR,ANN,XGB,y_test
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
995,1,1,1,1,1,1,1
996,1,1,1,1,1,1,1
997,1,1,1,1,1,1,1
998,1,1,1,1,1,1,1


In [41]:
newdata.columns

Index(['SVM', 'KNN', 'RF', 'LR', 'ANN', 'XGB', 'y_test'], dtype='object')

In [42]:
# Define the DNN model
DNNX = newdata.drop(columns=['y_test']).copy()
DNNY = newdata.y_test.copy()

model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='selu', input_shape=DNNX.shape[1:]),
    #tf.keras.layers.Dense(25, activation='selu'),
    #tf.keras.layers.Dense(20, activation='selu'),
    tf.keras.layers.Dense(15, activation='selu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(DNNX, DNNY, epochs=500, batch_size=128, validation_split=0.1)

Epoch 1/500
8/8 [==============================] - 1s 20ms/step - loss: 0.8035 - accuracy: 0.5256 - val_loss: 0.5970 - val_accuracy: 0.9700
Epoch 2/500
8/8 [==============================] - 0s 5ms/step - loss: 0.5492 - accuracy: 0.9544 - val_loss: 0.4348 - val_accuracy: 0.9700
Epoch 3/500
8/8 [==============================] - 0s 4ms/step - loss: 0.4338 - accuracy: 0.9578 - val_loss: 0.3640 - val_accuracy: 0.9700
Epoch 4/500
8/8 [==============================] - 0s 4ms/step - loss: 0.3710 - accuracy: 0.9600 - val_loss: 0.2992 - val_accuracy: 0.9700
Epoch 5/500
8/8 [==============================] - 0s 4ms/step - loss: 0.3146 - accuracy: 0.9600 - val_loss: 0.2538 - val_accuracy: 0.9600
Epoch 6/500
8/8 [==============================] - 0s 4ms/step - loss: 0.2707 - accuracy: 0.9600 - val_loss: 0.2186 - val_accuracy: 0.9600
Epoch 7/500
8/8 [==============================] - 0s 4ms/step - loss: 0.2342 - accuracy: 0.9600 - val_loss: 0.1908 - val_accuracy: 0.9600
Epoch 8/500
8/8 [=========

In [43]:
y_pred_DNN = (model.predict(DNNX) > 0.5).astype("int32")
Tr = model.evaluate(DNNX,DNNY,verbose=0)#[1]
Tr

32/32 [==============================] - 0s 830us/step


[0.10008710622787476, 0.968999981880188]

### **AutoML Individual and AutoML DNN**

In [44]:
acc['avg'] = (acc["train"] + acc['test'])/2
acc

,train,test,avg
SVM,0.958625,0.961,0.959812
KNN,0.958750,0.961,0.959875
RF,0.957500,0.962,0.959750
LR,0.960500,0.960,0.960250
ANN,0.967500,0.966,0.966750
XGB,0.990875,0.955,0.972938


In [45]:
#H2O AutoML

In [46]:
#!pip install h2o
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()
from h2o.model.segment_models import H2OFrame
from h2o.automl import H2OAutoML
print("All Library Loaded")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 43 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 24 days
H2O_cluster_name:,H2O_from_python_unknownUser_k02xfi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


All Library Loaded


In [52]:
hdf = h2o.H2OFrame(df)
hdf["diagnosis"] = hdf["diagnosis"].asfactor()
hdf

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
1,12.5179,14.6712,81.7703,504.688,0.0815631,0.0898908,0.0861605,0.0856867,0.159646,0.0591654,0.406238,0.950452,3.16915,43.4416,0.00638421,0.0185807,0.054688,0.0144285,0.0152963,0.00362328,15.9522,24.8596,154.63,589.112,0.145676,0.434579,0.429107,0.162828,0.25858,0.0736936
1,13.4334,24.134,88.0231,439.944,0.10021,0.158879,0.174838,0.0911813,0.18409,0.0618466,0.475145,2.02758,0.793628,49.5837,0.0083422,0.0287919,0.051537,0.0146678,0.0168495,0.00452331,16.63,32.7029,134.263,780.965,0.147551,0.385937,0.406677,0.209882,0.303234,0.111444
0,13.2587,13.7266,90.5187,550.735,0.0825516,0.043065,0.00223358,0,0.125201,0.0539142,0.267514,0.455168,1.37473,23.3505,0.00317751,0.00778596,0.00991266,0.00931237,0.0134395,0.0020002,13.9837,21.6937,103.421,731.55,0.10613,0.208155,0.0676216,0.0336224,0.250743,0.05504
0,19.9159,13.5563,112.658,1093.84,0.0895906,0.0400599,0.05132,0.0344726,0.139893,0.0550027,0.245379,0.712654,1.91677,13.9833,0.00537268,0.014095,0.0109191,0.0088618,0.0123678,0.00139395,26.0468,17.4086,85.4577,733.601,0.0967445,0.173591,0.128044,0.0651504,0.216223,0.0764176
0,10.1658,17.5137,69.3404,349.621,0.0823398,0.0743995,0.132373,0.0223626,0.160343,0.0609129,0.23037,1.62202,1.22429,14.6661,0.00616449,0.037435,0.0331937,0.0130135,0.0182423,0.00516332,12.9694,31.9539,74.3656,518.112,0.145097,0.15194,0.134512,0.0280096,0.291609,0.0736188
0,12.5147,17.2839,88.9711,423.877,0.0958337,0.15176,0.117529,0.0245492,0.186764,0.061305,0.15748,1.27017,1.43475,6.802,0.00656071,0.0355504,0.0463689,0.0132676,0.011993,0.00430502,14.7304,20.3284,76.9375,421.381,0.129793,0.253428,0.227938,0.0785696,0.276445,0.104647
0,11.2053,16.874,76.6924,306.963,0.107138,0.0721483,0.033077,0.0174127,0.181034,0.0619812,0.314161,0.734598,1.87183,22.8006,0.0102808,0.0105386,0.0278285,0.0135368,0.0193712,0.00447307,12.4045,20.7401,77.2059,368.693,0.149814,0.0915606,0.206482,0.0600058,0.332413,0.0758543
0,12.8983,19.6174,80.114,332.309,0.109391,0.144679,0.0841196,0.0298468,0.169397,0.059645,0.272769,1.3005,3.11382,35.5262,0.00538097,0.0397606,0.0336594,0.0123309,0.0156666,0.00392958,11.8968,29.6933,102.78,514.189,0.135774,0.177033,0.374084,0.0815227,0.220727,0.0771299
1,14.9404,26.5008,100.121,656.791,0.0816748,0.0782246,0.0743197,0.00809558,0.143851,0.04996,0.276563,0.876606,1.01405,18.7603,0.00404746,0.009617,0.0288984,0.0104577,0.0114481,0.00209546,24.3198,39.0515,133.855,2085.56,0.0957067,0.167878,0.33707,0.225486,0.219188,0.0606554
1,21.5391,18.0643,155.659,1170.13,0.0958501,0.154666,0.116588,0.0973235,0.148099,0.0550726,0.642793,0.890341,3.75224,69.5202,0.00360478,0.0154859,0.0258473,0.00600552,0.007882,0.00252777,25.031,26.9324,177.782,2421.67,0.146203,0.366781,0.264898,0.176736,0.335587,0.0809957


In [53]:
hy = "diagnosis"
hx = hdf.columns
hx.remove(hy)

In [54]:
train, valid = hdf.split_frame(ratios=[.8], seed=123)

In [56]:
aml = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=10,sort_metric='AUC')

In [57]:
aml.train(x = hx, y = hy, training_frame = train, validation_frame = valid)

AutoML progress: |
16:53:06.872: Project: AutoML_2_20230621_165306
16:53:06.872: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:53:06.874: Setting stopping tolerance adaptively based on the training frame: 0.011167086836213468
16:53:06.874: Build control seed: 10
16:53:06.874: training frame: Frame key: AutoML_2_20230621_165306_training_py_6_sid_9cfd    cols: 31    rows: 8019  chunks: 2    size: 1930731  checksum: 4588544707086808976
16:53:06.874: validation frame: Frame key: py_7_sid_9cfd    cols: 31    rows: 1981  chunks: 2    size: 480857  checksum: -506457625328556480
16:53:06.875: leaderboard frame: NULL
16:53:06.875: blending frame: NULL
16:53:06.875: response column: diagnosis
16:53:06.875: fold column: null
16:53:06.875: weights column: null
16:53:06.875: Loading exec

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/10
# GBM base models (used / total),2/4
# XGBoost base models (used / total),3/3
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,10
Metalearner fold_column,None


In [58]:
lb = aml.leaderboard
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_2_20230621_165306,0.995602,0.083005,0.993785,0.0364352,0.159777,0.0255287
StackedEnsemble_BestOfFamily_1_AutoML_2_20230621_165306,0.995601,0.0829281,0.993792,0.036902,0.159624,0.0254797
GBM_1_AutoML_2_20230621_165306,0.995148,0.0878287,0.993127,0.0395032,0.163783,0.0268249
GLM_1_AutoML_2_20230621_165306,0.995016,0.0882816,0.992998,0.0404367,0.165234,0.0273022
GBM_3_AutoML_2_20230621_165306,0.994998,0.0931254,0.992872,0.037317,0.16715,0.0279392
GBM_2_AutoML_2_20230621_165306,0.994863,0.0936337,0.99272,0.0411671,0.168463,0.0283798
XGBoost_3_AutoML_2_20230621_165306,0.994677,0.0954456,0.992402,0.0388669,0.169936,0.0288782
XGBoost_1_AutoML_2_20230621_165306,0.99456,0.095327,0.992335,0.0412693,0.169404,0.0286977
XGBoost_2_AutoML_2_20230621_165306,0.994557,0.0968987,0.992284,0.0406474,0.170343,0.0290166
GBM_4_AutoML_2_20230621_165306,0.994515,0.0974588,0.992239,0.041424,0.171064,0.0292628


In [59]:
best_model = aml.get_best_model()
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/10
# GBM base models (used / total),2/4
# XGBoost base models (used / total),3/3
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,10
Metalearner fold_column,None


In [61]:
best_model.model_performance(train).accuracy()

[[0.49935233486734654, 0.9910213243546577]]

In [65]:
best_model = aml.get_best_model()
HATr  = best_model.model_performance(train)
HATe  = best_model.model_performance(valid)

[[0.6035976176933681, 0.9565875820292782]]

In [68]:
acc = pd.DataFrame(
    {
    "SVM":[STr,STe],
    "KNN":[KTr,KTe],
    "RF" :[RTr,RTe],
    "LR" :[LTr,LTe],
    "ANN":[ATr,ATe],
    "XGB":[XTr,XTe],
    "H2OAutoML":[HATr.accuracy()[0][1],HATe.accuracy()[0][1]]
    })
acc.index = ["train", "test"]
acc = acc.T
acc

,train,test
SVM,0.958625,0.961000
KNN,0.958750,0.961000
RF,0.957500,0.962000
LR,0.960500,0.960000
ANN,0.967500,0.966000
XGB,0.990875,0.955000
H2OAutoML,0.991021,0.956588


In [ ]:
newdata = pd.DataFrame({
    "SVM"   : CVal(SVC(C=0.1, gamma='auto', kernel = 'rbf')),
    "KNN"   : CVal(KNeighborsClassifier(n_neighbors=3)),
    "RF"    : CVal(RandomForestClassifier(n_estimators= 500,max_features = 'sqrt', max_samples = 100, random_state=123)),
    'LR'    : CVal(LogisticRegression(C= 0.1 , penalty='l1', solver='liblinear', max_iter = 1000, random_state=123)),
    "ANN"   : CValANN(),
    "XGB"   : CVal(XGBClassifier(objective='binary:logistic',max_depth= 7,alpha= 10,learning_rate= 1,n_estimators=100)),
    'y_test': list(y_test)
})

In [ ]:
newdata["KNN"] = CVal(KNeighborsClassifier(n_neighbors=3))

In [ ]:
newdata["RF"]  = CVal(RandomForestClassifier(n_estimators= 500,max_features = 'sqrt', max_samples = 100, random_state=123))

In [ ]:
newdata['LR']  = CVal(LogisticRegression(C= 0.1 , penalty='l1', solver='liblinear', max_iter = 1000, random_state=123))

In [ ]:
newdata["ANN"] = CValANN()

In [ ]:
newdata["XGB"] = CVal(XGBClassifier(objective='binary:logistic',max_depth= 7,alpha= 10,learning_rate= 1,n_estimators=100))

In [ ]:
newdata['y_test'] = encoder.fit_transform(df['diagnosis']).copy()

In [ ]:
newdata

,SVM,KNN,RF,LR,ANN,XGB,y_test
0,1,1,1,1,1,0,1
1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1
4,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...
564,1,1,1,1,1,1,1
565,1,1,1,1,1,1,1
566,1,1,1,1,1,1,1
567,1,1,1,1,1,1,1
